# Double Robust DiD Estimation with Deep Learning

This notebook demonstrates how to use deep learning to estimate the average treatment effect in a difference-in-differences (DiD) setting.

Note:
- The code is quite computational extensive and may take a while to run.
- The code should be run on a GPU to speed up the training process.

In [ ]:
# Run the simulation and save the results
n_reps = 10  # change to 1000 for the full simulation
n_obs = 1000
true_att = 0.0
dgp_type = 4  # choose 1 to 4
ATTE = 0.0

In [ ]:
import warnings

import numpy as np
import tensorflow as tf
from doubleml import DoubleMLData, DoubleMLDID
from doubleml.datasets import make_did_SZ2020
from scikeras.wrappers import KerasClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

warnings.filterwarnings("ignore")
np.random.seed(42)

# Enable GPU acceleration if available
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)


def create_model(input_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))  # Assuming binary classification
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


def run_simulation(n_reps, n_obs, ATTE, dgp_type):
    # Initialize arrays to store statistics
    np.full(n_reps, np.nan)
    np.full(n_reps, np.nan)
    np.full(n_reps, np.nan)
    coverage = np.full(n_reps, np.nan)
    avg_variance = np.full(n_reps, np.nan)
    ci_length = np.full(n_reps, np.nan)
    ATTE_estimates = np.full(n_reps, np.nan)

    early_stopping = EarlyStopping(monitor="loss", patience=2, verbose=0)

    for i_rep in range(n_reps):
        x, y, d = make_did_SZ2020(
            n_obs=n_obs,
            dgp_type=dgp_type,
            cross_sectional_data=False,
            return_type="array",
        )
        dml_data = DoubleMLData.from_arrays(x=x, y=y, d=d)

        keras_classifier = KerasClassifier(
            build_fn=lambda: create_model(x.shape[1]),
            epochs=3,
            batch_size=32,
            verbose=0,
            callbacks=[early_stopping],
        )

        ml_m = Pipeline([("scaler", StandardScaler()), ("nn", keras_classifier)])
        ml_g = LinearRegression()

        dml_plr = DoubleMLDID(dml_data, ml_g, ml_m)
        dml_plr.fit()

        ATTE_estimates[i_rep] = dml_plr.coef.squeeze()
        confint = dml_plr.confint(level=0.95)
        coverage[i_rep] = (confint["2.5 %"].iloc[0] <= ATTE) & (
            confint["97.5 %"].iloc[0] >= ATTE
        )
        ci_length[i_rep] = confint["97.5 %"].iloc[0] - confint["2.5 %"].iloc[0]

        summary_df = dml_plr.summary
        std_err = summary_df.loc["d", "std err"]
        avg_variance[i_rep] = std_err**2

    avg_bias = np.mean(ATTE_estimates - ATTE)
    med_bias = np.median(ATTE_estimates - ATTE)
    rmse = np.sqrt(np.mean((ATTE_estimates - ATTE) ** 2))
    avg_variance = np.mean(avg_variance)
    coverage_probability = np.mean(coverage)
    avg_ci_length = np.mean(ci_length)

    results = {
        "avg_bias": avg_bias,
        "med_bias": med_bias,
        "rmse": rmse,
        "avg_variance": avg_variance,
        "coverage_probability": coverage_probability,
        "avg_ci_length": avg_ci_length,
    }

    # Ensure the directory exists
    latex_filename = f"bld/tables/dr_DL_sim_results_dgp_{dgp_type}.tex"

    # Writing the results to a LaTeX file
    with open(latex_filename, "w") as f:
        f.write("\\begin{table}[ht]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{|l|r|}\n")
        f.write("\\hline\n")
        f.write("Metric & Value \\\\\n")
        f.write("\\hline\n")
        for key, value in results.items():
            f.write(f"{key.replace('_', ' ').title()} & {value:.4f} \\\\\n")
        f.write("\\hline\n")
        f.write("\\end{tabular}\n")
        f.write(
            f"\\caption{{Simulation Results for double robust deep learning with DGP Type {dgp_type}}}\n",
        )
        f.write("\\end{table}\n")

    return results


results = run_simulation(n_reps, n_obs, ATTE, dgp_type)
print(results)

# IPW DiD Estimation with Deep Learning

In [ ]:
import numpy as np
import pandas as pd
from doubleml import DoubleMLData
from doubleml.datasets import make_did_SZ2020
from scipy.stats import iqr, norm
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Define std_ipw_did_rc function


def std_ipw_did_rc(
    y,
    post,
    D,
    covariates=None,
    i_weights=None,
    boot=False,
    boot_type="weighted",
    nboot=None,
    inffunc=False,
):
    # Convert inputs to numpy arrays
    D = np.asarray(D)
    n = len(D)
    y = np.asarray(y)
    post = np.asarray(post)

    # Add constant to covariate vector
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))

    # Weights
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        msg = "i.weights must be non-negative"
        raise ValueError(msg)
    i_weights = np.asarray(i_weights)

    # Pscore estimation (neural network) and fitted values
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(int_cov.shape[1],)))
    model.add(
        layers.Dense(10, activation="relu", kernel_regularizer=l2(0.01)),
    )  # Added L2 regularization
    model.add(layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(0.01)))

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    model.fit(int_cov, D, epochs=3, batch_size=32, verbose=0)

    ps_fit = model.predict(int_cov).flatten()
    ps_fit = np.clip(ps_fit, 1e-16, 1 - 1e-16)

    # Compute IPW estimator weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)

    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * y / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * y / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * y / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * y / np.mean(w_cont_post)

    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)

    # ATT estimator
    ipw_att = (att_treat_post - att_treat_pre) - (att_cont_post - att_cont_pre)

    # Influence function to compute standard error
    score_ps = i_weights[:, None] * (D - ps_fit)[:, None] * int_cov
    Hessian_ps = np.linalg.inv(np.dot(int_cov.T, score_ps))
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)

    # Influence function of the "treat" component
    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(
        w_treat_post,
    )
    inf_treat = inf_treat_post - inf_treat_pre

    # Influence function of the control component
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)
    inf_cont = inf_cont_post - inf_cont_pre

    # Estimation effect from gamma hat (pscore)
    M2_pre = np.mean(
        w_cont_pre[:, None] * (y - att_cont_pre)[:, None] * int_cov,
        axis=0,
    ) / np.mean(w_cont_pre)
    M2_post = np.mean(
        w_cont_post[:, None] * (y - att_cont_post)[:, None] * int_cov,
        axis=0,
    ) / np.mean(w_cont_post)
    inf_cont_ps = np.dot(asy_lin_rep_ps, (M2_post - M2_pre))

    # Influence function for the control component
    inf_cont += inf_cont_ps

    # Combine influence functions
    att_inf_func = inf_treat - inf_cont

    if not boot:
        # Standard error
        se_att = np.std(att_inf_func) / np.sqrt(n)
        uci = ipw_att + 1.96 * se_att
        lci = ipw_att - 1.96 * se_att
        ipw_boot = None
    else:
        if nboot is None:
            nboot = 999
        if boot_type == "multiplier":
            # Multiplier bootstrap
            ipw_boot = mboot_did(att_inf_func, nboot)
            se_att = iqr(ipw_boot) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.quantile(np.abs(ipw_boot / se_att), 0.95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att
        else:
            # Weighted bootstrap
            ipw_boot = [
                wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights)
                for _ in range(nboot)
            ]
            ipw_boot = np.array(ipw_boot)
            se_att = iqr(ipw_boot - ipw_att) / (norm.ppf(0.75) - norm.ppf(0.25))
            cv = np.quantile(np.abs((ipw_boot - ipw_att) / se_att), 0.95)
            uci = ipw_att + cv * se_att
            lci = ipw_att - cv * se_att

    if not inffunc:
        att_inf_func = None

    return {
        "ATT": ipw_att,
        "se": se_att,
        "uci": uci,
        "lci": lci,
        "boots": ipw_boot,
        "att_inf_func": att_inf_func,
    }


def mboot_did(att_inf_func, nboot):
    n = len(att_inf_func)
    boots = []
    for _ in range(nboot):
        weights = np.random.exponential(scale=1.0, size=n)
        boot_att = np.sum(weights * att_inf_func) / np.sum(weights)
        boots.append(boot_att)
    return np.array(boots)


def wboot_std_ipw_rc(n, y, post, D, int_cov, i_weights):
    indices = np.random.choice(n, n, replace=True)
    y_boot = y[indices]
    post_boot = post[indices]
    D_boot = D[indices]
    int_cov_boot = int_cov[indices]
    i_weights_boot = i_weights[indices]
    return std_ipw_did_rc(
        y_boot,
        post_boot,
        D_boot,
        int_cov_boot,
        i_weights_boot,
        boot=False,
    )["ATT"]


def perform_simulation(n_reps=10, n_obs=1000, true_att=0.0, dgp_type=1):
    att_estimates = []
    se_estimates = []
    uci_estimates = []
    lci_estimates = []

    for _ in range(n_reps):
        x, y, d = make_did_SZ2020(
            n_obs=n_obs,
            dgp_type=dgp_type,
            cross_sectional_data=False,
            return_type="array",
        )
        covariates = x
        post = np.concatenate([np.zeros(n_obs // 2), np.ones(n_obs // 2)])

        results = std_ipw_did_rc(y, post, d, covariates)

        att_estimates.append(results["ATT"])
        se_estimates.append(results["se"])
        uci_estimates.append(results["uci"])
        lci_estimates.append(results["lci"])

    att_estimates = np.array(att_estimates)
    se_estimates = np.array(se_estimates)
    uci_estimates = np.array(uci_estimates)
    lci_estimates = np.array(lci_estimates)

    # Calculate measures
    biases = att_estimates - true_att
    avg_bias = np.mean(biases)
    med_bias = np.median(biases)
    rmse = np.sqrt(np.mean(biases**2))
    var_att = np.var(att_estimates)
    avg_var = np.mean(var_att)
    ci_lengths = uci_estimates - lci_estimates
    coverage = np.mean((lci_estimates <= true_att) & (uci_estimates >= true_att))
    avg_ci_length = np.mean(ci_lengths)

    return {
        "Average Bias": avg_bias,
        "Median Bias": med_bias,
        "RMSE": rmse,
        "Average Variance of ATT": avg_var,
        "Coverage": coverage,
        "Confidence Interval Length": avg_ci_length,
    }


from pathlib import Path


def save_results_to_latex(measures, dgp_type):
    df = pd.DataFrame(list(measures.items()), columns=["Measure", "Value"])
    latex_filename = f"bld/ps_estim_DL_results/dr_PS_sim_results_dgp_{dgp_type}.tex"

    # Ensure the directory exists
    latex_path = Path(latex_filename)
    latex_path.parent.mkdir(parents=True, exist_ok=True)

    # Save the DataFrame to a LaTeX file
    df.to_latex(latex_filename, index=False)
    print(f"Results saved to {latex_filename}")


simulation_measures = perform_simulation(n_reps, n_obs, true_att, dgp_type)

# Print measures
for measure, value in simulation_measures.items():
    print(f"{measure}: {value}")

# Save the results to a LaTeX file
save_results_to_latex(simulation_measures, dgp_type)